# ETL Pipeline

## Extract - Tratamento de Campos

In [1]:
import pandas as pd
import pandera as pa
import regex as re

In [2]:
df = pd.read_csv("bolsistas_iniciacao_cientifica.csv", sep=';', encoding='UTF-8')

In [ ]:
df.head(10)

In [ ]:
df.shape

#### Ele se tornou float por ter nulos, porém neste tipo de dados temos que ter decimais, mas no banco ele é PK inteiro.

In [ ]:
# É necessário manter os NULOS para facilitar o processo de carga no Oracle
df1 = df['id_grupo_pesquisa'].astype(str)
df.id_grupo_pesquisa = df1.apply(lambda x: x.split('.')[0]).replace('nan', pd.NA)

### Tranformar a data Inicio e Fim, para apenas data, sem a Hora.

In [ ]:
df['inicio'] = pd.to_datetime(df['inicio']).dt.normalize()

In [ ]:
df['fim'] = pd.to_datetime(df['fim']).dt.normalize()

In [ ]:
df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.isnull().any()

#### Com o Pandera, é possível realizar um tratamento de atributos de forma mais prática

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "id_discente":pa.Column(pa.Int),
        "matricula":pa.Column(pa.Int),
        "discente":pa.Column(pa.String),
        "titulo":pa.Column(pa.Object, nullable=True),
        "codigo_projeto":pa.Column(pa.String),
        "id_projeto_pesquisa":pa.Column(pa.Int),
        "ano":pa.Column(pa.Int),
        "id_orientador":pa.Column(pa.Int),
        "orientador":pa.Column(pa.String),
        "categoria":pa.Column(pa.String),
        "tipo_de_bolsa":pa.Column(pa.String),
        "linha_pesquisa":pa.Column(pa.String, nullable=True),
        "id_grupo_pesquisa":pa.Column(pa.Object, nullable=True),
        "grupo_pesquisa":pa.Column(pa.String, nullable=True),
        "cota":pa.Column(pa.String),
        "inicio":pa.Column(pa.DateTime, nullable=True),
        "fim":pa.Column(pa.DateTime, nullable=True),
        "id_unidade":pa.Column(pa.Int),
        "unidade":pa.Column(pa.String),
        "status":pa.Column(pa.String)
    }
)

In [ ]:
schema.validate(df, inplace=True)

### Padronizar o nome das colunas para o mesmo definido na modelagem

In [ ]:
df = df.rename(columns={'id_projeto_pesquisa': 'cd_projeto_pesquisa', 'orientador': 'nm_orientador', 'tipo_de_bolsa'
                        : 'tipo_bolsa', 'id_grupo_pesquisa': 'id_grupo', 'grupo_pesquisa': 'nm_grupo', 'inicio'
                        : 'inicio_pesquisa', 'fim': 'fim_pesquisa', 'unidade': 'nm_unidade'})

#### Dropar a linha caso tenha a metade ou mais nulo

In [ ]:
df.dropna(axis=0, thresh=10, inplace=True)

## Transform - Tratamento dos Registros

### Vericar se contém caracteres especiais nos campos que não podem ter.

In [ ]:
df.loc[df.discente.notnull() & (df['discente'].astype(str).str.replace(" ", "").str.isalnum() == False)]

In [ ]:
# Realizar o tratamento para os valores anteriores
df.discente = df.discente.str.replace('.', '').str.replace('-', '')

In [ ]:
# Vericar se existem inconsistências no nome do nm_orientador

df.loc[df.nm_orientador.notnull() & (df['nm_orientador'].astype(str).str.replace(" ", "").str.isalnum() == False)]

In [ ]:
# Mudar o limite de linha e caracteres nos campos para facilitar a visualização dos próximos campos
pd.options.display.max_colwidth = 200

In [ ]:
# No caso do nm_grupo, não pode ter

df.loc[df.nm_grupo.notnull() & (df['nm_grupo'].astype(str).str.replace(" ", "").str.isalnum() == False)]

In [ ]:
# Mudar o limite de linha e caracteres nos campos para facilitar a visualização dos próximos 
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 50

### Tratar o nm_grupo, encontrado "." em alguns registros, devem ser compreendidos e retirados caso necessário

In [ ]:
# No caso do nm_grupo

df.loc[df.nm_grupo.notnull() & (df['nm_grupo'].str.contains(".", regex=False))]

In [ ]:
df.loc[df.nm_grupo.notnull() & df.nm_grupo.str.endswith('.'), 'nm_grupo']

#### Alguns registros, contém "." no final devem ser substituido por nada. 

In [ ]:
selecao = (df.nm_grupo.notnull()) & (df.nm_grupo.str.endswith('.'))
df.loc[selecao, 'nm_grupo'] = df.loc[selecao, 'nm_grupo'].str.replace('.', '', regex=False)
df

In [ ]:
df1.head(10)

#### Os grupos de pesquisa de ID 4878102 e 13087450, tem o ponto como vírgula

In [ ]:
selecao = (df.id_grupo.notnull()) & (df.id_grupo.str.match('(4878102)|(13087450)'))
df.loc[selecao, 'nm_grupo'] = df.loc[selecao, 'nm_grupo'].str.replace('.', ',', regex=False)

In [ ]:
df[df.id_grupo == '13087450']

In [ ]:
# Verificar as alterações, deve aparecer apenas um grupo

selecao = df.nm_grupo.notnull() & df['nm_grupo'].str.contains(".", regex=False)
df.loc[selecao, ['id_grupo', 'nm_grupo']].drop_duplicates()

### Verificar se existem caracteres não convertidos

In [ ]:
def col_caractere():
    col_list = []
    for col in df.columns: 
        try:
            mask = df[col].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
            if mask.any():
                col_list.append(col)
            else:
                continue
        except AttributeError:
            continue
    return col_list

In [ ]:
col_caractere()

#### Será necessário entender quais caracteres são cada um e em cada coluna que tem

In [ ]:
mask = df['titulo'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'titulo'].drop_duplicates()

In [ ]:
mask = df['linha_pesquisa'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'linha_pesquisa'].drop_duplicates()

In [ ]:
mask = df['nm_grupo'].fillna('').str.contains(r'[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~]')
df.loc[mask, 'nm_grupo'].drop_duplicates()

#### 1 - Todos os campo em que o caractere está entre dois espaços deveria ser o meio hífen, porém poderá ser transformado em hífen. 

#### 2 - Se estiverem com o \t na frente, deve ser substituido por "- "

#### Percorrerá todas as colunas que tem o caractere.

In [ ]:
regex = ['[\s][^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\s]', '[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\t]']

mask = df['nm_grupo'].fillna('').str.contains(r'{}'.format(regex[1]))
df.loc[mask, 'nm_grupo'].drop_duplicates()

In [ ]:
cols = col_caractere()
regex = ['[\s][^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\s]', '[^\w\s\x00-\x7F!"\#$%&()*+,\-./:;<=>?@^_‘{|}~][\t]']

for i in cols:
    df[i] = df[i].str.replace(regex[0], ' - ',  regex=True).str.replace(regex[1], '- ', regex=True)

### Restaram apenas os caracteres que ficam no começo e fim de cada texto, esses serão apagados conforme a RN 20

In [ ]:
col_list = []

for col in df.columns: 
    try:
        mask = df[col].fillna('').str.contains(r'^[^\w\x00-\x7F\d(]+|[^\w\x00-\x7F\d)]+$')
        if mask.any():
            col_list.append(col)
        else:
            continue
    except AttributeError:
        continue

In [ ]:
col_list

In [ ]:
for col in col_list:
    df[col] = df[col].str.replace(r'^[^\x00-\x7F\d]+|[^\x00-\x7F\d]+$', '', regex=True)

### Alguns textos não seguem o padrão de Português para os sinais de pontuação. Alguns, não tem um espaço depois, dificultando a visualização, por exemplo: 'texto;texto', sendo que deveria ser 'texto; texto'.

In [ ]:
col_list = []

for col in df.columns: 
    try:
        mask = df[col].fillna('').str.contains(r'[\w][,:;!?][\w]')
        if mask.any():
            col_list.append(col)
        else:
            continue
    except AttributeError:
        continue

In [ ]:
col_list

In [ ]:
mask = df['titulo'].fillna('').str.contains(r'[\w][,:;!?][\w]')
df.loc[mask, 'titulo'].drop_duplicates()

In [ ]:
mask = df['linha_pesquisa'].fillna('').str.contains(r'[\w][,:;!?][\w]')
df.loc[mask, 'linha_pesquisa'].drop_duplicates()

In [ ]:
mask = df['nm_grupo'].fillna('').str.contains(r'[\w][,:;!?][\w]')
df.loc[mask, 'nm_grupo'].drop_duplicates()

### Cada uma das colunas retornadas pelo loop, seguem a sua diferença, no caso do título, a vírgula não tem espaço por conta da forma de escrita da Química

In [ ]:
df['titulo'] = df['titulo'].fillna('').str.replace(r'(\w)(:;!?)(\w)', '\1\2 \3', regex=True)

In [ ]:
df['titulo'] = df['titulo'].fillna('').str.replace(r'(\w)(,:;!?)(\w)', '\1\2 \3', regex=True)

In [ ]:
df['linha_pesquisa'] = df['linha_pesquisa'].fillna('').str.replace(r'(\w)(,:;!?)(\w)', '\1\2 \3', regex=True)